<a href="https://colab.research.google.com/github/OwiyeD/Datawarehousing/blob/master/Analyzing_fortunate_500_companies_from_Wikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mining data from wikipedia

In [ ]:
#%logstop
#%logstart -rtq ~/.logs/DS_Data_Munging.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
import pandas as pd
import json
df = pd.read_html('https://en.wikipedia.org/w/index.php?title=Fortune_Global_500&oldid=855890446', header=0)[0]
fortune_500 = json.loads(df.to_json(orient="records"))
df

,Rank,Company,Country,Industry,Revenue in USD
0,1,Walmart,United States,Retail,$500 billion
1,2,State Grid,China,Power,$349 billion
2,3,Sinopec Group,China,Petroleum,$327 billion
3,4,China National Petroleum,China,Petroleum,$326 billion
4,5,Royal Dutch Shell,Netherlands,Petroleum,$312 billion
5,6,Toyota Motor,Japan,Automobiles,$265 billion
6,7,Volkswagen,Germany,Automobiles,$260 billion
7,8,BP,United Kingdom,Petroleum,$245 billion
8,9,Exxon Mobil,United States,Petroleum,$244 billion
9,10,Berkshire Hathaway,United States,Products,$242 billion


Lets see our data in a form of a list of dictionaries

In [ ]:
fortune_500

[{'Company': 'Walmart',
  'Country': 'United States',
  'Industry': 'Retail',
  'Rank': 1,
  'Revenue in USD': '$500 billion'},
 {'Company': 'State Grid',
  'Country': 'China',
  'Industry': 'Power',
  'Rank': 2,
  'Revenue in USD': '$349 billion'},
 {'Company': 'Sinopec Group',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Rank': 3,
  'Revenue in USD': '$327 billion'},
 {'Company': 'China National Petroleum',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Rank': 4,
  'Revenue in USD': '$326 billion'},
 {'Company': 'Royal Dutch Shell',
  'Country': 'Netherlands',
  'Industry': 'Petroleum',
  'Rank': 5,
  'Revenue in USD': '$312 billion'},
 {'Company': 'Toyota Motor',
  'Country': 'Japan',
  'Industry': 'Automobiles',
  'Rank': 6,
  'Revenue in USD': '$265 billion'},
 {'Company': 'Volkswagen',
  'Country': 'Germany',
  'Industry': 'Automobiles',
  'Rank': 7,
  'Revenue in USD': '$260 billion'},
 {'Company': 'BP',
  'Country': 'United Kingdom',
  'Industry': 'Petroleum',
  'Ra

This is a great start to our analysis, however, there really isn't that much information here, we will need to bring in additional data sources to get any further understanding of these companies.

The first question we might want to ask is how many employees does it take to get that revenue, in other words, what is the revenue per employee? Luckily, we can use Wikipedia to get that data as well, we have scraped this data manually (all from Wikipedia) and created the following dictionary.

In [ ]:
other_data = [
    {"name": "Walmart",
     "employees": 2300000,
     "year founded": 1962
    },
    {"name": "State Grid Corporation of China",
     "employees": 927839,
     "year founded": 2002},
    {"name": "China Petrochemical Corporation",
     "employees":358571,
     "year founded": 1998
     },
    {"name": "China National Petroleum Corporation",
     "employees": 1636532,
     "year founded": 1988},
    {"name": "Toyota Motor Corporation",
     "employees": 364445,
     "year founded": 1937},
    {"name": "Volkswagen AG",
     "employees": 642292,
     "year founded": 1937},
    {"name": "Royal Dutch Shell",
     "employees": 92000,
     "year founded": 1907},
    {"name": "Berkshire Hathaway Inc.",
     "employees":377000,
     "year founded": 1839},
    {"name": "Apple Inc.",
     "employees": 123000,
     "year founded": 1976},
    {"name": "Exxon Mobile Corporation",
     "employees": 69600,
     "year founded": 1999},
    {"name": "BP plc",
     "employees": 74000,
     "year founded": 1908}
]


Some data have a slightly different name than in our original set, so we will keep a dictionary of mappings between the two. Notice, we only include the mapping in the dictionary if there is a difference

In [ ]:
mapping = {
    'Apple': 'Apple Inc.',
    'BP': 'BP plc',
    'Berkshire Hathaway': 'Berkshire Hathaway Inc.',
    'China National Petroleum': 'China National Petroleum Corporation',
    'Exxon Mobil': 'Exxon Mobile Corporation',
    'Sinopec Group': 'China Petrochemical Corporation',
    'State Grid': 'State Grid Corporation of China',
    'Toyota Motor': 'Toyota Motor Corporation',
    'Volkswagen': 'Volkswagen AG'
}


This data is one to one, meaning the data contained in one source only aligns with a single element in the other source, thus we should be able to put these together. However, we know that the data isn't in a great form to be joined at the moment. This is for two reasons:

1. All the names will not align (we need to use our mapping)
2. The list structure is not optimized for looking through elements.

While for 10 elements the second reason won't really matter, for larger data sets such performance considerations are extremely important. We can turn this list of dictionaries into a dictionary of dictionaries, so we can quickly access each element of the data.

In [ ]:

dict_data = {k["name"] : k for k in other_data}
dict_data

{'Apple Inc.': {'employees': 123000,
  'name': 'Apple Inc.',
  'year founded': 1976},
 'BP plc': {'employees': 74000, 'name': 'BP plc', 'year founded': 1908},
 'Berkshire Hathaway Inc.': {'employees': 377000,
  'name': 'Berkshire Hathaway Inc.',
  'year founded': 1839},
 'China National Petroleum Corporation': {'employees': 1636532,
  'name': 'China National Petroleum Corporation',
  'year founded': 1988},
 'China Petrochemical Corporation': {'employees': 358571,
  'name': 'China Petrochemical Corporation',
  'year founded': 1998},
 'Exxon Mobile Corporation': {'employees': 69600,
  'name': 'Exxon Mobile Corporation',
  'year founded': 1999},
 'Royal Dutch Shell': {'employees': 92000,
  'name': 'Royal Dutch Shell',
  'year founded': 1907},
 'State Grid Corporation of China': {'employees': 927839,
  'name': 'State Grid Corporation of China',
  'year founded': 2002},
 'Toyota Motor Corporation': {'employees': 364445,
  'name': 'Toyota Motor Corporation',
  'year founded': 1937},
 'Volksw


Question: If we had many entries in other_data, we could display a small piece by printing other_data[:5]. With dataframes we might use df.head(). Can you think of a way to print out a small piece of a dictionary?

Now we can easily compute the revenue per employee, we need to map the "Company" value in our original data with the "name" column of this other data, but we also need to use the mapping to ensure the columns will line up. We in general don't want to mutate our original data, so lets make a new list of dictionaries with this new feature (revenue per employee). On the course of doing this, we will need to handle converting some numbers like $500 Billion to a numeric value. Lets create a function to do this.

In [ ]:

def convert_revenue(x):
    return float(x.lstrip('$').rstrip('billion')) * 1e9

assert convert_revenue('$500 billion') == 500e9

# functions to compute this revenue per employee and create a data list

In [ ]:
def rev_per_emp(company):
    name = company[u'Company']
    n_employees = dict_data[mapping.get(name, name)].get('employees')
    company['rev per emp'] = convert_revenue(company[u'Revenue in USD'])/n_employees
    return company

def compute_copy(d, func):
    return func({k:v for k,v in d.items()})

data = list(map(lambda x : compute_copy(x, rev_per_emp), fortune_500))

Sort the values

In [ ]:
rev_per_emp = sorted([(i[u'Company'], i['rev per emp']) for i in data], 
                   key=lambda x : x[1],
                   reverse=True)
rev_per_emp

[('Exxon Mobil', 3505747.1264367816),
 ('Royal Dutch Shell', 3391304.347826087),
 ('BP', 3310810.810810811),
 ('Sinopec Group', 911953.2812190612),
 ('Toyota Motor', 727133.0379069544),
 ('Berkshire Hathaway', 641909.8143236075),
 ('Volkswagen', 404800.30889377417),
 ('State Grid', 376142.84374767606),
 ('Walmart', 217391.30434782608),
 ('China National Petroleum', 199201.72657790987)]

lets pull in some other data. We can see that these companies are in a few different industries, let find out which ones.

In [ ]:
from collections import Counter
Counter(i[u'Industry'] for i in data)

Counter({'Automobiles': 2,
         'Petroleum': 5,
         'Power': 1,
         'Products': 1,
         'Retail': 1})

One thing we might want to know is what sort of market share they have of the specific industry to which they belong. Let's look at the two industries that categorize the 6 of the top 10, Automobiles and Petroleum. We can select only those elements of our data to work with.

In [ ]:
sub_data = [i for i in data if i[u'Industry'] in [u'Automobiles', u'Petroleum']]
sub_data

[{'Company': 'Sinopec Group',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Rank': 3,
  'Revenue in USD': '$327 billion',
  'rev per emp': 911953.2812190612},
 {'Company': 'China National Petroleum',
  'Country': 'China',
  'Industry': 'Petroleum',
  'Rank': 4,
  'Revenue in USD': '$326 billion',
  'rev per emp': 199201.72657790987},
 {'Company': 'Royal Dutch Shell',
  'Country': 'Netherlands',
  'Industry': 'Petroleum',
  'Rank': 5,
  'Revenue in USD': '$312 billion',
  'rev per emp': 3391304.347826087},
 {'Company': 'Toyota Motor',
  'Country': 'Japan',
  'Industry': 'Automobiles',
  'Rank': 6,
  'Revenue in USD': '$265 billion',
  'rev per emp': 727133.0379069544},
 {'Company': 'Volkswagen',
  'Country': 'Germany',
  'Industry': 'Automobiles',
  'Rank': 7,
  'Revenue in USD': '$260 billion',
  'rev per emp': 404800.30889377417},
 {'Company': 'BP',
  'Country': 'United Kingdom',
  'Industry': 'Petroleum',
  'Rank': 8,
  'Revenue in USD': '$245 billion',
  'rev per emp': 3310810

It might be the case that the each particular category has a different relevant metric for market share. For example, we could look at total revenue for a car company or we could look at cars produced.

So for the automobile industry we will look at the percent total of cars produced. We can get this data again from Wikipedia.

In [ ]:
df_list = pd.read_html("https://en.wikipedia.org/w/index.php?title=Automotive_industry&oldid=875776152", header=0)
car_totals = json.loads(df_list[0].to_json(orient="records"))
car_by_man = json.loads(df_list[2].to_json(orient='records'))

In [ ]:
car_totals[:2]

[{'Change': '—', 'Production': 54434000, 'Source': '[17]', 'Year': 1997},
 {'Change': '2.7%', 'Production': 52987000, 'Source': '[17]', 'Year': 1998}]

In [ ]:
car_by_man[:2]

[{'Country': 'Japan', 'Group': 'Toyota', 'Rank': 1, 'Vehicles': 10213486},
 {'Country': 'Germany',
  'Group': 'Volkswagen Group',
  'Rank': 2,
  'Vehicles': 10126281}]

Now lets get only the groups we care about and divide by the total production which we will take as the latest year.

In [ ]:
total_prod = sorted((i[u"Year"], i[u'Production']) for i in car_totals)[-1][1]
total_prod

97302534

Now we can find the market share for each of the car companies. We will keep track of a market share dictionary. We will again need to keep track of some slight name differences.

In [ ]:
car_by_man_dict = {i[u'Group']:i[u'Vehicles'] for i in car_by_man}
market_share = {}
for name, orig_name in zip(['Toyota', 'Volkswagen Group'], ['Toyota', 'Volkswagen']):
    market_share[orig_name] = car_by_man_dict[name]/ float(total_prod)
    
market_share

{'Toyota': 0.1049662899837737, 'Volkswagen': 0.10407006460900597}


Now we can do the same for the Petroleum industry, but in this case, lets compute the market share by revenue. On Wikipedia, we can find a list of oil companies by revenue. Although its not a complete list, it has enough companies that we don't expect the companies left off the list to contribute greatly to our analysis.

In [ ]:

rev = pd.read_html("https://en.wikipedia.org/w/index.php?title=List_of_largest_oil_and_gas_companies_by_revenue&oldid=871711850", header=1)[0]
rev = rev.iloc[:, 1:3]
rev.columns = ['Company', 'Revenue']
rev = rev[~(rev['Company'] == 'Company name')]
oil_data = json.loads(rev.to_json(orient="records"))
oil_data[:2]

[{'Company': 'Saudi Aramco', 'Revenue': '465.49'},
 {'Company': 'Sinopec Group', 'Revenue': '448.00'}]


Now we can compute the totals and market share. Since the data here might be slightly different (perhaps older) than our original data, we will compute the market share of each company within this data set, then pull out the numbers we care about.

In [ ]:
total = sum([float(i[u'Revenue'].rstrip('*')) for i in oil_data])
shares = {i[u'Company']:float(i[u'Revenue'].rstrip('*'))/total for i in oil_data}
print(total)

5482.090000000001



Now we can pull out the companies we care about in the petroleum industry.

In [ ]:
petro_companies = [i[u'Company'] for i in data if i['Industry'] == u'Petroleum']
petro_companies

['Sinopec Group',
 'China National Petroleum',
 'Royal Dutch Shell',
 'BP',
 'Exxon Mobil']

Lets check if these are all in the our shares dictionary.

In [ ]:
[(i, i in shares) for i in petro_companies]

[('Sinopec Group', True),
 ('China National Petroleum', False),
 ('Royal Dutch Shell', True),
 ('BP', True),
 ('Exxon Mobil', True)]

Some of these companies are directly there, and looking through our dictionary, we can see the others are there without exact names

In [ ]:
shares.keys()

dict_keys(['Saudi Aramco', 'Sinopec Group', 'China National Petroleum Corporation', 'Exxon Mobil', 'Royal Dutch Shell', 'Kuwait Petroleum Corporation', 'BP', 'Total SA', 'Lukoil', 'Eni', 'Valero Energy', 'Chevron Corporation', 'PDVSA', 'Pemex', 'National Iranian Oil Company', 'Gazprom', 'Phillips 66', 'Petronas', 'China National Offshore Oil', 'Marathon Petroleum', 'PTT', 'Rosneft', 'JX Holdings', 'Engie', 'Petrobras', 'Equinor', 'Indian Oil Corporation', 'Sonatrach', 'Reliance Industries', 'Pertamina', 'SOCAR', 'GS Caltex', 'Enterprise Products', 'Repsol', 'Centrica', 'Bharat Petroleum', 'OMV Group', 'Idemitsu Kosan', 'Hindustan Petroleum', 'PKN Orlen', 'Schlumberger', 'Suncor Energy', 'Hellenic Petroleum', 'ConocoPhillips', 'Motor Oil Hellas', 'Ecopetrol', 'CEPSA', 'MOL'])

So lets make a fuzzy match, this will be a pretty simple one where it will try to match words in a name and take the maximum number of matches.

In [ ]:

def fuzzy_match(word, s):
    words = set(word.split(' '))
    overlaps = [(k, len(v.intersection(words))) for k, v in s.items()]
    return max(overlaps, key=lambda x : x[1])[0]

In [ ]:
split_names = {i: set(i.split(' ')) for i in shares.keys()}
for i in petro_companies:
    match = fuzzy_match(i, split_names)
    print("matched {} to {}".format(i, match))
    market_share[i] = shares[match]

matched Sinopec Group to Sinopec Group
matched China National Petroleum to China National Petroleum Corporation
matched Royal Dutch Shell to Royal Dutch Shell
matched BP to BP
matched Exxon Mobil to Exxon Mobil


In [ ]:
market_share

{'BP': 0.040641434197541446,
 'China National Petroleum': 0.07818550954106918,
 'Exxon Mobil': 0.0490506357976611,
 'Royal Dutch Shell': 0.04833922828702191,
 'Sinopec Group': 0.0817206576323993,
 'Toyota': 0.1049662899837737,
 'Volkswagen': 0.10407006460900597}